In [26]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split

data_loc = "data/titanic/"
titanic = pd.read_csv(data_loc + "titanic.csv")

titanic.head()

c:\users\matth\appdata\local\programs\python\python35\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [27]:
titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [28]:
# Convert sex to a binary value of either 1 for male or 0 for female
def filter_sex(df):
    sex_series = pd.Series( np.where( df.Sex == 'male' , 1 , 0 ) , name = 'Sex' )
    return sex_series

In [29]:
print(filter_sex(titanic).head())

0    1
1    0
2    0
3    0
4    1
Name: Sex, dtype: int32


In [30]:
# Fill missing values of Age with the average of Age (mean)
def filter_age(df):
    age_df = pd.DataFrame() # Create a new dataframe to store our results
    age_df[ 'Age' ] = df.Age.fillna(df.Age.mean())
    return age_df

In [31]:
print(filter_age(titanic).head())

    Age
0  22.0
1  38.0
2  26.0
3  35.0
4  35.0


In [32]:
# Fill missing values of fare with the average of the fare (mean)
def filter_fare(df):
    fare_df = pd.DataFrame() # Create a new dataframe to store our results
    fare_df['fare'] = df.Fare.fillna(df.Fare.mean())
    return fare_df

In [33]:
print(filter_fare(titanic).head())

      fare
0   7.2500
1  71.2833
2   7.9250
3  53.1000
4   8.0500


In [34]:
def filter_title(df):
    title_df = df[ 'Name' ].map( lambda name: name.split( ',' )[1].split( '.' )[0].strip() )

    # a map of more aggregated titles
    Title_Dictionary = {
                        "Capt":       "Officer",
                        "Col":        "Officer",
                        "Major":      "Officer",
                        "Jonkheer":   "Royalty",
                        "Don":        "Royalty",
                        "Sir" :       "Royalty",
                        "Dr":         "Officer",
                        "Rev":        "Officer",
                        "the Countess":"Royalty",
                        "Dona":       "Royalty",
                        "Mme":        "Mrs",
                        "Mlle":       "Miss",
                        "Ms":         "Mrs",
                        "Mr" :        "Mr",
                        "Mrs" :       "Mrs",
                        "Miss" :      "Miss",
                        "Master" :    "Master",
                        "Lady" :      "Royalty"

                        }

    # we map each title
    title_df = title_df.map( Title_Dictionary )
    title_df = pd.get_dummies( title_df )
    return title_df

In [35]:
print(filter_title(titanic).head())

   Master  Miss  Mr  Mrs  Officer  Royalty
0       0     0   1    0        0        0
1       0     0   0    1        0        0
2       0     1   0    0        0        0
3       0     0   0    1        0        0
4       0     0   1    0        0        0


In [36]:
def filter_cabin(df):
    cabin_df = pd.DataFrame()

    # replacing missing cabins with U (for Uknown)
    cabin_df[ 'Cabin' ] = df.Cabin.fillna( 'U' )

    # mapping each Cabin value with the cabin letter
    cabin_df[ 'Cabin' ] = cabin_df[ 'Cabin' ].map( lambda c : c[0] )

    # dummy encoding ...
    cabin_df = pd.get_dummies( cabin_df['Cabin'] , prefix = 'Cabin' )
    return cabin_df

In [37]:
print(filter_cabin(titanic).head())

   Cabin_A  Cabin_B  Cabin_C  Cabin_D  Cabin_E  Cabin_F  Cabin_G  Cabin_T  \
0        0        0        0        0        0        0        0        0   
1        0        0        1        0        0        0        0        0   
2        0        0        0        0        0        0        0        0   
3        0        0        1        0        0        0        0        0   
4        0        0        0        0        0        0        0        0   

   Cabin_U  
0        1  
1        0  
2        1  
3        0  
4        1  


In [38]:
def cleanTicket( ticket ):
    ticket = ticket.replace( '.' , '' )
    ticket = ticket.replace( '/' , '' )
    ticket = ticket.split()
    ticket = map( lambda t : t.strip() , ticket )
    ticket = list(filter( lambda t : not t.isdigit() , ticket ))
    if len( ticket ) > 0:
        return ticket[0]
    else: 
        return 'XXX'

def filter_ticket(df):
    ticket_df = pd.DataFrame()

    # Extracting dummy variables from tickets:
    ticket_df[ 'Ticket' ] = df[ 'Ticket' ].map( cleanTicket )
    ticket_df = pd.get_dummies( ticket_df[ 'Ticket' ] , prefix = 'Ticket' )
    
    return ticket_df

In [39]:
print(filter_ticket(titanic).head())

   Ticket_A4  Ticket_A5  Ticket_AS  Ticket_C  Ticket_CA  Ticket_CASOTON  \
0          0          1          0         0          0               0   
1          0          0          0         0          0               0   
2          0          0          0         0          0               0   
3          0          0          0         0          0               0   
4          0          0          0         0          0               0   

   Ticket_FC  Ticket_FCC  Ticket_Fa  Ticket_LINE     ...      Ticket_SOPP  \
0          0           0          0            0     ...                0   
1          0           0          0            0     ...                0   
2          0           0          0            0     ...                0   
3          0           0          0            0     ...                0   
4          0           0          0            0     ...                0   

   Ticket_SOTONO2  Ticket_SOTONOQ  Ticket_SP  Ticket_STONO  Ticket_STONO2  \
0        

In [40]:
def filter_family(df):
    family_df = pd.DataFrame()

    # introducing a new feature : the size of families (including the passenger)
    family_df[ 'FamilySize' ] = df[ 'Parch' ] + df[ 'SibSp' ] + 1

    # introducing other features based on the family size
    family_df[ 'Family_Single' ] = family_df[ 'FamilySize' ].map( lambda s : 1 if s == 1 else 0 )
    family_df[ 'Family_Small' ]  = family_df[ 'FamilySize' ].map( lambda s : 1 if 2 <= s <= 4 else 0 )
    family_df[ 'Family_Large' ]  = family_df[ 'FamilySize' ].map( lambda s : 1 if 5 <= s else 0 )
    return family_df

In [41]:
print(filter_family(titanic).head())

   FamilySize  Family_Single  Family_Small  Family_Large
0           2              0             1             0
1           2              0             1             0
2           1              1             0             0
3           2              0             1             0
4           1              1             0             0


In [42]:
def filter_survived(df):
    df[ 'Survived' ].map( lambda s : 1 if s == 1 else 0 )

In [43]:
# Now mash it all together into one big dataframe
def format_data(df):
    sex_series = filter_sex(df)
    age_df = filter_age(df)
    fare_df = filter_fare(df)
    title_df = filter_title(df)
    cabin_df = filter_cabin(df)
    ticket_df = filter_ticket(df)
    family_df = filter_family(df)
    
    x_data = pd.concat( [age_df, fare_df, title_df, cabin_df, ticket_df, family_df, sex_series ] , axis=1 )
    y_data = df.Survived
    return x_data, y_data

In [44]:
print(format_data(titanic)[1].head())

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64


In [109]:
import tensorflow as tf

data_x, data_y = format_data(titanic)
train_x , test_x , train_y , test_y = train_test_split( data_x , data_y , train_size = .8 )

learning_rate = 0.01
epochs = 20

input_nodes = 53
output_nodes = 1
batch_size = 10

x = tf.placeholder('float', [None, input_nodes])
y = tf.placeholder('float', [None, output_nodes])

hidden_1_nodes = 100
hidden_2_nodes = 50

hlw_1 = tf.Variable(tf.random_normal([input_nodes, hidden_1_nodes])) # The weights of the first hidden layer
hlb_1 = tf.Variable(tf.random_normal([hidden_1_nodes])) # The biases of the first hidden layer

hlw_2 = tf.Variable(tf.random_normal([hidden_1_nodes, hidden_2_nodes])) # The weights of the second hidden layer
hlb_2 = tf.Variable(tf.random_normal([hidden_2_nodes])) # The biases of the second hidden layer

outw = tf.Variable(tf.random_normal([hidden_2_nodes, output_nodes])) # The weights of the output layer
outb = tf.Variable(tf.random_normal([output_nodes])) # The biases of the output layer

In [110]:
def forward_propagation(x):
    hlout_1 = tf.nn.sigmoid(tf.matmul(x, hlw_1) + hlb_1) # input multiplied by hidden layer 1 weights. Add bias. Apply sigmoid
    hlout_2 = tf.nn.sigmoid(tf.matmul(hlout_1, hlw_2) + hlb_2) # hidden layer 1 output multiplied by hidden layer 2 weights. Add bias. Apply sigmoid
    output = tf.matmul(hlout_2, outw) + outb # hidden layer 2 output multiplied by output layer weights. add bias. DON'T add sigmoid
    return output

In [111]:
def backward_propagation(y_, y, learning_rate):
    cost = tf.reduce_mean( tf.nn.sigmoid_cross_entropy_with_logits(logits=y_, labels=y)) #Error/loss function. Cost is the total error over the batch
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost) #Adam optimizer is a type of gradient descent. We want it to use the learning rate and minimize that cost
    return optimizer, cost

In [112]:
def calculate_accuracy(y_, y):
    correct = tf.equal(tf.round(y_), tf.round(y)) # argmax finds the index of the largest value in a tensor. So we're comparing the two using tf.equal
    accuracy = tf.reduce_mean(tf.cast(correct, 'float')) # We've got a tensor of true and false statements now. Let's find the average amount
    return accuracy, correct

In [113]:
y_ = forward_propagation(x)
optimizer, cost = backward_propagation(y_, y, learning_rate)
accuracy = calculate_accuracy(y_, y)

In [114]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

cost_per_epoch = [] 
accuracy_per_epoch = []

# Loop over the dataset 'epochs' amount of times
for epoch in range(epochs):
    epoch_cost = 0
    
    # Go through the dataset in batches
    for i in range(int(len(train_x)/batch_size)):
        batch_x = train_x[i*batch_size:i*batch_size + batch_size]
        batch_y = train_y[i*batch_size:i*batch_size + batch_size]
        
        batch_x = np.reshape(batch_x, [len(batch_x), input_nodes])
        batch_y = np.reshape(batch_y, [len(batch_y), output_nodes])
        
        _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y}) # Run our forward prop and backward prop over this batch of training data
        epoch_cost += c
    
    # Test the accuracy of our model against the training set
    epoch_accuracy, _ = sess.run(accuracy, feed_dict={x: np.reshape(train_x, [len(train_x), input_nodes]), y: np.reshape(train_y, [len(train_y), output_nodes])}) # Pass all the labels through to calculate accuracy
    print('Epoch', epoch + 1, 'completed out of', epochs, '\ncost:', epoch_cost, '\ntraining accuracy:', epoch_accuracy, '\n')
    
    # Store the results of this epoch so we can make nice graphs later
    cost_per_epoch.append(epoch_cost)
    accuracy_per_epoch.append(epoch_accuracy) 

c:\users\matth\appdata\local\programs\python\python35\lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


Epoch 1 completed out of 20 
cost: 50.1654184163 
training accuracy: 0.273876 

Epoch 2 completed out of 20 
cost: 37.0381807834 
training accuracy: 0.241573 

Epoch 3 completed out of 20 
cost: 34.78424187 
training accuracy: 0.151685 

Epoch 4 completed out of 20 
cost: 32.6923056692 
training accuracy: 0.158708 

Epoch 5 completed out of 20 
cost: 32.0743770823 
training accuracy: 0.186798 

Epoch 6 completed out of 20 
cost: 30.270070076 
training accuracy: 0.144663 

Epoch 7 completed out of 20 
cost: 29.589460142 
training accuracy: 0.0828652 

Epoch 8 completed out of 20 
cost: 29.195765309 
training accuracy: 0.103933 

Epoch 9 completed out of 20 
cost: 28.4211011827 
training accuracy: 0.155899 

Epoch 10 completed out of 20 
cost: 29.0015235171 
training accuracy: 0.125 

Epoch 11 completed out of 20 
cost: 28.6256000027 
training accuracy: 0.109551 

Epoch 12 completed out of 20 
cost: 28.4734841064 
training accuracy: 0.113764 

Epoch 13 completed out of 20 
cost: 28.03591